In [13]:
import numpy as np
import pandas as pd
import os

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from scipy.special import rel_entr
from scipy.stats import entropy
import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

In [14]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [15]:
! nvidia-smi

Tue Jun 14 13:33:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 26%   34C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 24%   

# Company數量

In [16]:
from pymongo import MongoClient
import pymongo

In [17]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_de = db.patent_de
    db_us = db.patent_us
    db_cn = db.patent_cn
except errors.ConnectionFailure as err:
    print(err)

In [18]:
data_assignee = db_de.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [19]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count']).head(15)

In [20]:
df_family_de

,company,count
0,Intel Corp,442
1,Samsung Electronics Co Ltd,147
2,Intel IP Corp,89
3,Apple Inc,73
4,Robert Bosch GmbH,58
5,LG Electronics Inc,38
6,Siemens AG,30
7,Fraunhofer Gesellschaft zur Forderung der Ange...,28
8,Deutsche Telekom AG,24
9,Sony Corp,24


In [21]:
data_assignee = db_cn.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count']).head(15)

In [22]:
df_family_cn

,company,count
0,Qualcomm Inc,2696
1,Samsung Electronics Co Ltd,2229
2,Huawei Technologies Co Ltd,1832
3,Telefonaktiebolaget LM Ericsson AB,900
4,Shenzhen Shenglu IoT Communication Technology ...,656
5,Intel Corp,636
6,Shanghai Langbo Communication Technology Co Ltd,597
7,Apple Inc,513
8,Xian Irain IoT Technology Service Co Ltd,500
9,China Mobile Communications Group Co Ltd,433


In [23]:
data_assignee = db_us.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count']).head(15)

In [39]:
df_family_us

,company,count
0,Qualcomm Inc,5795
1,Samsung Electronics Co Ltd,3961
2,Intel Corp,2158
3,Telefonaktiebolaget LM Ericsson AB,1588
4,International Business Machines Corp,1569
5,LG Electronics Inc,1511
6,Cisco Technology Inc,1092
7,Microsoft Technology Licensing LLC,748
8,Huawei Technologies Co Ltd,629
9,AT&T Intellectual Property I LP,540


In [40]:
com_ls_de = list(df_family_de.company)

In [41]:
com_ls_cn = list(df_family_cn.company)

In [42]:
com_ls_us = list(df_family_us.company)

# CPC data

In [33]:
us_5 = pd.read_csv("./rm_company_cpc_us_5.csv")
us_10 = pd.read_csv("./rm_company_cpc_us_10.csv")

In [34]:
us_all = pd.concat([us_5,us_10], join='outer')

In [35]:
de_5 = pd.read_csv("./rm_company_cpc_de_us_5.csv")
de_10 = pd.read_csv("./rm_company_cpc_de_us_10.csv")

In [36]:
de_all = pd.concat([de_5,de_10], join='outer')

In [37]:
cn_5 = pd.read_csv("./rm_company_cpc_cn_us_5.csv")
cn_10 = pd.read_csv("./rm_company_cpc_cn_us_10.csv")

In [38]:
cn_all = pd.concat([cn_5,cn_10], join='outer')

## US

In [54]:
cpc_ls = ['H04', 'G06', 'H01', 'G01', 'A61', 'Y02', 'G08', 'B60', 'G05', 'H03']

In [52]:
us_df = us_all[us_all.company.isin(com_ls_us)].groupby("company", axis=0).sum()
us_df

,A01,A23,A41,A42,A44,A46,A47,A61,A62,A63,...,E21,F02,F17,F22,F23,F26,F27,F41,F42,G12
company,,,,,,,,,,,,,,,,,,,,,
AT&T Intellectual Property I LP,0.0,1.0,0.0,0.0,0.0,0.0,0.0,56.0,0.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cisco Technology Inc,0.0,0.0,0.0,0.0,0.0,0.0,6.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Huawei Technologies Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Intel Corp,14.0,0.0,5.0,0.0,0.0,2.0,6.0,128.0,0.0,64.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
International Business Machines Corp,21.0,0.0,0.0,0.0,0.0,0.0,28.0,395.0,2.0,48.0,...,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
LG Electronics Inc,0.0,0.0,0.0,3.0,0.0,0.0,136.0,309.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Micron Technology Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Microsoft Technology Licensing LLC,1.0,0.0,2.0,0.0,0.0,0.0,0.0,49.0,0.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
us_df.loc[:,cpc_ls].reset_index()

,company,H04,G06,H01,G01,A61,Y02,G08,B60,G05,H03
0,AT&T Intellectual Property I LP,3722.0,492.0,5.0,53.0,56.0,35.0,133.0,15.0,10.0,10.0
1,Apple Inc,5432.0,189.0,28.0,107.0,0.0,78.0,10.0,0.0,6.0,27.0
2,Cisco Technology Inc,6511.0,1595.0,29.0,82.0,16.0,118.0,40.0,58.0,50.0,5.0
3,Huawei Technologies Co Ltd,3949.0,197.0,51.0,37.0,0.0,59.0,36.0,14.0,5.0,59.0
4,Intel Corp,10334.0,4566.0,2274.0,342.0,128.0,310.0,194.0,85.0,148.0,204.0
5,International Business Machines Corp,3361.0,5705.0,1005.0,266.0,395.0,131.0,293.0,235.0,170.0,34.0
6,LG Electronics Inc,9141.0,2555.0,48.0,282.0,309.0,129.0,194.0,1278.0,408.0,72.0
7,Micron Technology Inc,525.0,2848.0,59.0,15.0,0.0,45.0,22.0,50.0,22.0,83.0
8,Microsoft Technology Licensing LLC,2090.0,3154.0,51.0,106.0,49.0,33.0,23.0,8.0,34.0,9.0
9,Nokia Technologies Oy,3158.0,293.0,6.0,60.0,18.0,26.0,31.0,0.0,10.0,18.0


## DE

In [59]:
cpc_ls = ['H04', 'G06', 'H01', 'G01', 'A61', 'Y02', 'G08', 'B60', 'G05', 'H03']

In [60]:
de_df = de_all[de_all.company.isin(com_ls_de)].groupby("company", axis=0).sum()
de_df

,A01,A23,A41,A42,A44,A46,A47,A61,A62,A63,...,E21,F02,F17,F22,F23,F26,F27,F41,F42,G12
company,,,,,,,,,,,,,,,,,,,,,
Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Deutsche Telekom AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fraunhofer Gesellschaft zur Forderung der Angewandten Forschung eV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Giesecke and Devrient Mobile Security GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hewlett Packard Enterprise Development LP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hyundai Motor Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Intel Corp,14.0,0.0,5.0,0.0,0.0,2.0,6.0,151.0,0.0,110.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Intel IP Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
International Business Machines Corp,21.0,0.0,1.0,0.0,0.0,0.0,28.0,395.0,2.0,48.0,...,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
de_df.loc[:,cpc_ls].reset_index()

,company,H04,G06,H01,G01,A61,Y02,G08,B60,G05,H03
0,Apple Inc,6119.0,191.0,28.0,109.0,0.0,86.0,10.0,0.0,6.0,27.0
1,Deutsche Telekom AG,289.0,27.0,0.0,8.0,0.0,8.0,7.0,2.0,5.0,0.0
2,Fraunhofer Gesellschaft zur Forderung der Ange...,790.0,15.0,60.0,21.0,4.0,6.0,0.0,0.0,3.0,11.0
3,Giesecke and Devrient Mobile Security GmbH,95.0,49.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,Hewlett Packard Enterprise Development LP,1106.0,431.0,5.0,16.0,0.0,14.0,10.0,0.0,3.0,20.0
5,Hyundai Motor Co,795.0,211.0,32.0,36.0,83.0,38.0,67.0,786.0,27.0,6.0
6,Intel Corp,11613.0,5572.0,2367.0,373.0,151.0,351.0,202.0,87.0,150.0,301.0
7,Intel IP Corp,780.0,36.0,2.0,25.0,0.0,21.0,0.0,0.0,1.0,57.0
8,International Business Machines Corp,3406.0,5791.0,1063.0,284.0,395.0,133.0,294.0,235.0,170.0,34.0
9,LG Electronics Inc,9821.0,2670.0,48.0,283.0,309.0,146.0,210.0,1283.0,408.0,72.0


## CN

In [62]:
cpc_ls = ['H04', 'G06', 'H01', 'G01', 'A61', 'Y02', 'G08', 'B60', 'G05', 'H03']

In [63]:
cn_df = cn_all[cn_all.company.isin(com_ls_cn)].groupby("company", axis=0).sum()
cn_df

,A01,A23,A41,A42,A43,A44,A45,A46,A47,A61,...,C13,C30,D05,D07,E06,F22,F26,F41,F42,G12
company,,,,,,,,,,,,,,,,,,,,,
Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chengdu Qinchuan IoT Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
China Mobile Communications Group Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Huawei Technologies Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Intel Corp,14.0,0.0,5.0,0.0,3.0,0.0,0.0,2.0,6.0,145.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jiangsu IoT Research and Development Center,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LG Electronics Inc,0.0,0.0,0.0,3.0,1.0,0.0,2.0,0.0,140.0,326.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NTT Docomo Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Qualcomm Inc,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,25.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
cn_df.loc[:,cpc_ls].reset_index()

,company,H04,G06,H01,G01,A61,Y02,G08,B60,G05,H03
0,Apple Inc,7435.0,233.0,28.0,118.0,0.0,98.0,10.0,0.0,6.0,35.0
1,Chengdu Qinchuan IoT Technology Co Ltd,273.0,190.0,0.0,43.0,0.0,36.0,19.0,45.0,9.0,0.0
2,China Mobile Communications Group Co Ltd,2011.0,275.0,22.0,47.0,0.0,102.0,56.0,8.0,21.0,0.0
3,Huawei Technologies Co Ltd,11445.0,523.0,73.0,70.0,0.0,192.0,89.0,28.0,22.0,160.0
4,Intel Corp,11684.0,6068.0,2436.0,366.0,145.0,353.0,205.0,87.0,156.0,251.0
5,Jiangsu IoT Research and Development Center,82.0,216.0,192.0,15.0,29.0,16.0,8.0,0.0,10.0,0.0
6,LG Electronics Inc,10923.0,2686.0,48.0,294.0,326.0,156.0,210.0,1288.0,416.0,72.0
7,NTT Docomo Inc,3716.0,4.0,1.0,4.0,0.0,30.0,7.0,0.0,0.0,0.0
8,Qualcomm Inc,50089.0,1537.0,2009.0,781.0,32.0,516.0,168.0,29.0,75.0,1076.0
9,Samsung Electronics Co Ltd,27636.0,10895.0,3027.0,903.0,863.0,475.0,305.0,232.0,309.0,813.0
